In [1]:
import pandas as pd
from tabulate import tabulate
import json
Cart = pd.DataFrame()


inventory = pd.read_csv('inventory.csv')

# Dictionary to store total bills for each phone number
customer_bills = {}
# Dictionary to store whether coupons were applied for each phone number
customer_coupons_applied = {}

# Define functions for user options
def view_inventory():
    print(tabulate(inventory, headers='keys', tablefmt='fancy_grid'))

def update_items():
    global inventory
    item_number = int(input("Enter item number: "))
    if item_number in inventory['Item Number'].values:
        new_quantity = int(input("Enter new quantity: "))
        new_price = float(input("Enter new price: "))
        inventory.loc[inventory['Item Number'] == item_number, 'Quantity'] = new_quantity
        inventory.loc[inventory['Item Number'] == item_number, 'Price'] = new_price
    else:
        item_name = input("Enter item name: ")
        new_item = pd.DataFrame({'Item Number': [item_number], 'Item Name': [item_name], 'Quantity': [int(input("Enter quantity: "))], 'Price': [float(input("Enter price: "))]})
        inventory = pd.concat([inventory, new_item], ignore_index=True)
    # Update inventory CSV file
    inventory.to_csv('inventory.csv', index=False)

def modify_quantity():
    item_number = int(input("Enter item number: "))
    if item_number in inventory['Item Number'].values:
        new_quantity = int(input("Enter new quantity: "))
        inventory.loc[inventory['Item Number'] == item_number, 'Quantity'] = new_quantity
    else:
        print("Item not found in inventory.")
    # Update inventory CSV file
    inventory.to_csv('inventory.csv', index=False)

# Define functions for customer options
def view_inventory_customer():
    print(tabulate(inventory, headers='keys', tablefmt='fancy_grid'))

def add_to_cart():
    global Cart
    item_number = int(input("Enter item number: "))
    if item_number in inventory['Item Number'].values:
        quantity = int(input("Enter quantity: "))
        if inventory.loc[inventory['Item Number'] == item_number, 'Quantity'].values[0] < quantity:
            print("Not enough stock available.")
        else:
            item_name = inventory.loc[inventory['Item Number'] == item_number, 'Item Name'].values[0]
            item_price = inventory.loc[inventory['Item Number'] == item_number, 'Price'].values[0]
            cart_item = pd.DataFrame({'Item Number': [item_number], 'Item Name': [item_name], 'Quantity': [quantity], 'Price': [item_price]})
            if Cart.empty:
                Cart = cart_item
            else:
                Cart = pd.concat([Cart, cart_item], ignore_index=True)
            # Update inventory after adding items to the cart
            inventory.loc[inventory['Item Number'] == item_number, 'Quantity'] -= quantity
            # Update inventory CSV file
            inventory.to_csv('inventory.csv', index=False)
            print("Item added to cart.")
    else:
        print("Item not found in inventory.")

def apply_coupons(total_price, phone_number):
    # Logic to apply coupons and return discounted price
    # For demonstration, let's assume a fixed discount of 10%
    if phone_number in customer_bills:
        previous_total = customer_bills[phone_number]
        if previous_total >= 100 and not customer_coupons_applied.get(phone_number, False):  # Check if previous bill qualifies for coupons and coupons not already applied
            print("Congratulations! You have earned coupons for your previous purchase.")
            total_price -= 10  # For demonstration, applying a fixed discount of 10
            print("Coupon Applied: Rs. 10 discount")
            customer_coupons_applied[phone_number] = True  # Mark coupons applied for this phone number
    
    # Update the total bill for the phone number
    customer_bills[phone_number] = total_price
    
    # Store customer information in a JSON file
    customer_info = {
        "phone_number": phone_number,
        "bill_amount": total_price,
        "coupons_applied": customer_coupons_applied.get(phone_number, False)
    }
    with open("customer_info.json", "a") as f:
        json.dump(customer_info, f)
        f.write("\n")
    
    return total_price

def process_payment(amount_due):
    payment_method = input("Select payment method (Cash/Card/UPI): ")
    if payment_method.lower() == "cash":
        cash_received = float(input("Enter amount received: "))
        if cash_received >= amount_due:
            change = cash_received - amount_due
            print("Payment successful. Change: Rs.", change)
        else:
            print("Insufficient amount received. Payment failed.")
    elif payment_method.lower() == "card" or payment_method.lower() == "upi":
        print("Payment successful.")
    else:
        print("Invalid payment method. Payment failed.")

def bill():
    global Cart
    if Cart.empty:
        print("Cart is empty.")
    else:
        total_price_before_coupons = 0
        print("SH Mart".center(40, '-'))
        print("{:<15} {:<20} {:<10} {:<10}".format("Item Number", "Item Name", "Price", "Quantity"))
        for i, row in Cart.iterrows():
            item_number = row['Item Number']
            item_name = row['Item Name']
            item_price = row['Price']
            quantity = row['Quantity']
            total_price_before_coupons += item_price * quantity
            print("{:<15} {:<20} {:<10} {:<10}".format(item_number, item_name, item_price, quantity))
        
        phone_number = input("Enter your phone number: ")
        total_price_after_coupons = apply_coupons(total_price_before_coupons, phone_number)
        
        print("-" * 40)
        print("{:<45} {:.2f}".format("Total Price (Before Coupons):", total_price_before_coupons))
        if total_price_after_coupons != total_price_before_coupons:
            print("{:<45} {:.2f}".format("Total Price (After Coupons):", total_price_after_coupons))
        print("-" * 40)
        
        process_payment(total_price_after_coupons)
        
        # Reset Cart for the next person
        Cart = pd.DataFrame()

# Main program loop
while True:
    print("\nWelcome to Supermarket\n1. User\n2. Customer\n3. Exit")
    choice = int(input("Enter your choice: "))
    if choice == 1:
        while True:
            print("\nUser Options\n1. View Inventory\n2. Update Items\n3. Modify Quantity\n4. Exit")
            user_choice = int(input("Enter your choice: "))
            if user_choice == 1:
                view_inventory()
            elif user_choice == 2:
                update_items()
            elif user_choice == 3:
                modify_quantity()
            elif user_choice == 4:
                break
    elif choice == 2:
        while True:
            print("\nCustomer Options\n1. View Inventory\n2. Add to Cart\n3. Bill\n4. Exit")
            customer_choice = int(input("Enter your choice: "))
            if customer_choice == 1:
                view_inventory_customer()
            elif customer_choice == 2:
                add_to_cart()
            elif customer_choice == 3:
                bill()
            elif customer_choice == 4:
                break
            else:
                print("Invalid choice. Please enter a valid option.")
    elif choice == 3:
        print("Visit Again. Thank you!")
        break
    else:
        print("Invalid choice. Please enter a valid option.")



Welcome to Supermarket
1. User
2. Customer
3. Exit
Enter your choice: 1

User Options
1. View Inventory
2. Update Items
3. Modify Quantity
4. Exit
Enter your choice: 1
╒════╤═══════════════╤═════════════╤════════════╤═════════╕
│    │   Item Number │ Item Name   │   Quantity │   Price │
╞════╪═══════════════╪═════════════╪════════════╪═════════╡
│  0 │             1 │ milk        │        100 │      25 │
├────┼───────────────┼─────────────┼────────────┼─────────┤
│  1 │             2 │ bread       │        138 │      30 │
├────┼───────────────┼─────────────┼────────────┼─────────┤
│  2 │             3 │ butter      │        421 │      30 │
├────┼───────────────┼─────────────┼────────────┼─────────┤
│  3 │             4 │ jam         │        788 │      30 │
├────┼───────────────┼─────────────┼────────────┼─────────┤
│  4 │             5 │ maggie      │        600 │      30 │
├────┼───────────────┼─────────────┼────────────┼─────────┤
│  5 │             6 │ juice       │        327 │  